Load documents
chunk documents
Clean up docs from irrelevant text
create a vector database from documents
create a Retriever
create a retriever engine from and retriever and knowledge base
write a prompt with context from retriever engine and user query
load pretrained model and embeddings
pass prompt to pretrained model



In [1]:
! pip install llama_index==0.10.19 llama_index_core==0.10.19 torch llama-index-embeddings-huggingface peft optimum bitysandbytes

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.2/15.3 MB 4.6 MB/s eta 0:00:04
     - -------------------------------------- 0.7/15.3 MB 9.2 MB/s eta 0:00:02
     ---- ----------------------------------- 1.7/15.3 MB 13.3 MB/s eta 0:00:02
     -------- ------------------------------- 3.2/15.3 MB 20.5 MB/s eta 0:00:01
     ----------- ---------------------------- 4.5/15.3 MB 22.1 MB/s eta 0:00:01
     ---------------- ----------------------- 6.3/15.3 MB 25.4 MB/s eta 0:00:01
     -------------------- ------------------- 7.8/15.3 MB 27.9 MB/s eta 0:00:01
     ------------------------- -------------- 9.7/15.3 MB 29.4 MB/s eta 0:00:01
     ---------------------------- ---------- 11.4/15.3 MB 38.5 MB/s eta 0:00:01
     --------------------------------- ----- 13.3/15.3 MB 40.9 MB/s eta 0:00:01
     ------------------------------------- - 14.9/15.3 MB 43.7 MB/s eta 0:00:01
     --------------------------------------  15.3/

ERROR: Could not find a version that satisfies the requirement bitysandbytes (from versions: none)
ERROR: No matching distribution found for bitysandbytes

[notice] A new release of pip is available: 23.0.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine 
from llama_index.core.postprocessor import SimilarityPostprocessor
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# to globally set whatever resources we are going to use
# go to hugging face, search models(embeddings)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.llm = None 
# if we have 5000 word document, if chunk size is 256, 5000/256 = 20 chunks
Settings.chunk_size = 256
# content of last 15 words of first chunk == content of first 15 word of second chunk; to avoid loosing context we use chunk overlap
Settings.chunk_overlap = 15

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\AppData\Local\llama_index\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


LLM is explicitly disabled. Using MockLLM.


In [4]:
documents = SimpleDirectoryReader("content1").load_data()

print(len(documents))

for doc in documents:
    if len(doc.text) == 0:
        documents.remove(doc)
        continue
print(len(documents))    

48
48


In [5]:
# creating a vector store

index = VectorStoreIndex.from_documents(documents)

# set number of docs to retrieve
# top 2 documents will be retrieved from the topic from which we ask question
top_k = 2

# Configure the retriever

retriever = VectorIndexRetriever(
    index = index,
    similarity_top_k = top_k,

)

In [7]:
# Assembling the query engine
# Retriever will retrive the answers from the index we have created
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    # her, we are keeping similarity cut-off(dcuments which are 50% similar will be queried by the retriever queryengine and from those top2 will be retrived )
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)]

)

In [8]:
query = "What's all this text about?"

response = query_engine.query(query)

print(response)

Context information is below.
---------------------
page_label: 30
file_path: c:\Users\DELL\Rag_app_llama\content1\2308.12950v3.pdf

We use a
special instruction to help models understand the specific question format: “read from and write to standard
IO” for standard questions and “use the provided function signature” for call-based questions, which we insert
into our prompt as the question guidance. Despite not finetuned on the training data nor provided with few
30

page_label: 29
file_path: c:\Users\DELL\Rag_app_llama\content1\2308.12950v3.pdf

(2021), which is comprised of a set of middle-school math word problems. Results are
summarised on Table 12.
29
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What's all this text about?
Answer: 


Here, we are getting certain vague answer


In [9]:
query = "What's all this text about?"

response = query_engine.query(query)

print(response)

context = "Context: \n"

for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

Context information is below.
---------------------
page_label: 30
file_path: c:\Users\DELL\Rag_app_llama\content1\2308.12950v3.pdf

We use a
special instruction to help models understand the specific question format: “read from and write to standard
IO” for standard questions and “use the provided function signature” for call-based questions, which we insert
into our prompt as the question guidance. Despite not finetuned on the training data nor provided with few
30

page_label: 29
file_path: c:\Users\DELL\Rag_app_llama\content1\2308.12950v3.pdf

(2021), which is comprised of a set of middle-school math word problems. Results are
summarised on Table 12.
29
---------------------
Given the context information and not prior knowledge, answer the query.
Query: What's all this text about?
Answer: 
Context: 
We use a
special instruction to help models understand the specific question format: “read from and write to standard
IO” for standard questions and “use the provided function signature

It is giving answers from 2 pages from our document.

Now, we will be doing prompt engineering to get proper answer

In [10]:
# Load model
# we will filter model for text generation in hugging face
# loading a normal huggingface model is like this
#model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# mistral is a gated model, we need to ask for permission for using that.
model_name = "Qwen/Qwen2.5-1.5B-Instruct" # 4gb model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code = False,
    revision = "main",
    #device_map = 'cuda:0' # when we try to load model on GPU. we have CPU, so comment this line.
    ) 

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

c:\Users\DELL\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--Qwen--Qwen2.5-1.5B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


--instruct model in Qwen are specifically fine tuned for chat purpose, without instruct , its core model, its good for finetuning from scratch

In [13]:
prompt_template_with_context = lambda context, query : f"""you are an AI assistant tasked with answering question based on the provided PDF content.
Please analyze the following excerpt from the PDF and answer the question.
PDF content:
{context}

Question : {query}

Instructions:
- Answer only based on the information provided in the PDF content above.
- If the answer cannot be found in the provided content, say " I cannot find the answer to the question and provide a pdf documents"
- BE concise and specific.
- Include relevant quote or references from the PDF when applicable.

Answer:
"""

In [16]:
# now, we need to write a good prompt
query = "What is the text about ?"
# in prompt , we will be passing prompt template with context
prompt = prompt_template_with_context(context,query)
inputs = tokenizer(prompt, return_tensors='pt' )
outputs = model.generate(input_ids = inputs["input_ids"], max_new_tokens = 280)
print(tokenizer.batch_decode(outputs)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


you are an AI assistant tasked with answering question based on the provided PDF content.
Please analyze the following excerpt from the PDF and answer the question.
PDF content:
Context: 
We use a
special instruction to help models understand the specific question format: “read from and write to standard
IO” for standard questions and “use the provided function signature” for call-based questions, which we insert
into our prompt as the question guidance. Despite not finetuned on the training data nor provided with few
30

(2021), which is comprised of a set of middle-school math word problems. Results are
summarised on Table 12.
29



Question : What is the text about ?

Instructions:
- Answer only based on the information provided in the PDF content above.
- If the answer cannot be found in the provided content, say " I cannot find the answer to the question and provide a pdf documents"
- BE concise and specific.
- Include relevant quote or references from the PDF when applicable.

An

The text discusses the use of special instructions for model understanding in question formats like standard and call-based questions, using the example of a middle-school math problem dataset called 30 (2021). The methods involve inserting specific prompts into the question to guide model responses effectively. Based on this summary, it seems that the focus was on refining how models interpret different types of input questions rather than directly addressing the topic itself. Therefore, while the passage touches on the application of certain techniques in handling specific kinds of inputs, it does not explicitly define what the text is about but instead describes how to approach such problems within those contexts. To directly address what the text is about without referring back to the original document would require more context about its intended audience or purpose beyond merely describing the methodological framework being applied. Thus, the best concise answer here would be: "I cannot find the answer to the question and provide a pdf documents"

In [17]:
# now, we need to write a good prompt
query = "What is long context fine tuning ?"
# in prompt , we will be passing prompt template with context
prompt = prompt_template_with_context(context,query)
inputs = tokenizer(prompt, return_tensors='pt' )
outputs = model.generate(input_ids = inputs["input_ids"], max_new_tokens = 280)
print(tokenizer.batch_decode(outputs)[0])

you are an AI assistant tasked with answering question based on the provided PDF content.
Please analyze the following excerpt from the PDF and answer the question.
PDF content:
Context: 
We use a
special instruction to help models understand the specific question format: “read from and write to standard
IO” for standard questions and “use the provided function signature” for call-based questions, which we insert
into our prompt as the question guidance. Despite not finetuned on the training data nor provided with few
30

(2021), which is comprised of a set of middle-school math word problems. Results are
summarised on Table 12.
29



Question : What is long context fine tuning ?

Instructions:
- Answer only based on the information provided in the PDF content above.
- If the answer cannot be found in the provided content, say " I cannot find the answer to the question and provide a pdf documents"
- BE concise and specific.
- Include relevant quote or references from the PDF when appli

you are an AI assistant tasked with answering question based on the provided PDF content.
Please analyze the following excerpt from the PDF and answer the question.
PDF content:
Context: 
We use a
special instruction to help models understand the specific question format: “read from and write to standard
IO” for standard questions and “use the provided function signature” for call-based questions, which we insert
into our prompt as the question guidance. Despite not finetuned on the training data nor provided with few
30

(2021), which is comprised of a set of middle-school math word problems. Results are
summarised on Table 12.
29



Question : What is long context fine tuning ?

Instructions:
- Answer only based on the information provided in the PDF content above.
- If the answer cannot be found in the provided content, say " I cannot find the answer to the question and provide a pdf documents"
- BE concise and specific.
- Include relevant quote or references from the PDF when applicable.

Answer:
Based on the provided content, long context fine tuning refers to the process of fine-tuning a model's understanding of a larger context beyond just reading and writing standard IO instructions. The method involves using additional data that isn't typically present during training, such as the provided function signatures for call-based questions. This approach helps improve the model's ability to handle more complex tasks without being explicitly trained on the same dataset used for initial training. However, this method wasn't applied to the given training data (30) due to its limited size. The results were summarized in Table 12 but no direct definition of long context fine tuning was provided within the text. To directly address what long context fine tuning means, you would need to refer back to the original source where it was introduced.<|endoftext|>

Improvements:
1. Documents Processing:
* Multiple document support
* Different formats(pdf, docx, txt, markdown)
* Table and Image extraction
* OCR integration
* Document metadata extraction and utilization

2. Enhanced retrieval features:
* Hybrid Search (Combining dense and sparse retrievers)
* Cross document reference linking
* semantic chunking instead of fixed size chunk
* dynamic Chunk size based on content structures
* Re- ranking(top_k) of retrieved passages
* Citation tracking and source attribution

3. User experience feature
* Chat history
* Document summary generation
* question based on document content
* Interactive document exploration
* Confidence scores for Answers
* Real time answer streaming
* Multi - lingual support

4. Answer Enhancements
* Answer Verification against source
* Multiple answer generation with voting

5. Performance Optimization
* Caching mechanism for frequently asked queries
* Batch- Processing of multiple questions.
* GPU acceleration
* Incremental Indexing for large documents
* Query Processing and Optimization
* Dynamic model selection based on query complexity.(which model do you wish to use? QWEn, llama , GPT)

6. Security and Privacy features
* Document access control
* User authentication
* Data Retention Policies
* PII detection and Handling

7. UI/UX Enhancement side
* Mobile Optimization
* Progress  indicators(( instead of spinner, progress bar can be added))
* Source highlighting in context.

8. Customization options
* Custom Prompt template
* Retrieval Strategy selection

** integrate groq in RAG

